In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.5.3`
import $ivy.`edu.berkeley.cs::chiseltest:0.5.3`
import $plugin.$ivy.`edu.berkeley.cs:chisel3-plugin_2.12.15:3.5.3`
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test
import scala.language.reflectiveCalls

## Problem 1  - Scala Loops
> To familiarize ourselves with looping in Scala, we will attempt a variant of the game [_Fizz buzz_](https://en.wikipedia.org/wiki/Fizz_buzz):

* _*Count*_ from 1 until `max` (exclusive)
* Maintain three sums: `fizz`, `buzz`, and `fizzbuzz`
* If the current count is divisible by ...
    * _15_ $\rightarrow$ add count to `fizzbuzz`
    * _5_ $\rightarrow$ add count to `buzz`
    * _3_ $\rightarrow$ add count to `fizz`
    * Add to the sums with the precedence rules given, so for example, if the count is _15_, only add to `fizzbuzz`.

In [ ]:
def fizzbuzz(max: Int): (Int, Int, Int) = {
    var (fizz, buzz, fizzbuzz) = (0, 0, 0)

    // YOUR CODE HERE
    ???
}

fizzbuzz(20)

In [ ]:
assert(fizzbuzz(2) == (0, 0, 0))
assert(fizzbuzz(20) == (48, 15, 15))
assert(fizzbuzz(35) == (153, 60, 45))


## Problem 2 - Using a Vec to Implement a Lookup Table
> Vecs allow us to dynamically index into a Chisel collection during operation in the generated hardware.

> In the problem, we use a Vec to implement a _lookup table_, which is a read-only memory (ROM) that holds precomputed results. The `SquareTable` module we provide below produces the result of squaring its input. However, the generated hardware contains only the lookup table because the multiplication is done at generation time. As a first implementation, we hardcode the IO widths and table size.

> Fill in the tester `testSquareTable` to exhaustively test all of `SquareTable`'s inputs (hint: there are 32).

In [ ]:
class SquareTable extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(5.W))
        val xSquared = Output(UInt(10.W))
    })
    // 0.U, 1.U, ..., 31.U
    val romData = Seq.tabulate(32)(i => (i*i).U(10.W))
    show(s"romData: $romData")

    val ROM: Vec[UInt] = VecInit(romData)
    io.xSquared := ROM(io.x)
}

In [ ]:
def testSquareTable: Boolean = {
    test(new SquareTable) { dut =>
        
        // YOUR CODE HERE
        ???
    }
    true
}

In [ ]:
assert(testSquareTable)

## Problem 3 - Parameterizing SquareTable
> In this problem, we will revise the module `SquareTable` from the previous problem to make the number of entries parameterized. First, ensure your tester `testSquareTable` is complete and correct first because it will help you with this problem too. Write a `SquareTable2` module below that takes a Scala Int `nEntries` as a parameter. `SquareTable2` will generate a lookup table (read-only UInts stored as a `Vec`) of nEntries entries. The output `xSquared` should be the result of squaring the input `x`. The table will support inputs from 0 to nEntries-1 (inclusive). Ensure the widths are correct, namely that x is wide enough to handle all inputs (hint: consider log2Ceil), and xSquared is sufficiently wide to not loose any data. `SquareTable2` behaves the same as `SquareTable`, except its size is a configurable parameter. You will also need to revise your tester (create `testSquareTable2`) so it is parameterized too.

In [ ]:
// YOUR CODE HERE
???

In [ ]:
def testSquareTable2: Boolean = {
    for (nEntries <- Seq(4, 5, 10, 30)) {
        test(new SquareTable2(nEntries)) { dut =>

            // YOUR CODE HERE
            ???
        }
    }
    true
}

In [ ]:
assert(testSquareTable2)


## Problem 4 - Scala functions
> We can write Chisel logic within a Scala function and use them within our modules.

> Write a Scala function named `sum` that takes two UInts as arguments and returns the sum (with width growth). You might find the [Chisel Cheat Sheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf) a helpful reference.

In [ ]:
// YOUR CODE HERE
???

In [ ]:
def testSum(): Boolean = {
    test(new Module {
            val io = IO(new Bundle {
                val in1 = Input(UInt(4.W))
                val in2 = Input(UInt(4.W))
                val out = Output(UInt())
            })
            io.out := sum(io.in1, io.in2)
        }) { dut => 
            for (in1 <- 0 until 16) {
                for (in2 <- 0 until 16) {
                    dut.io.in1.poke(in1.U)
                    dut.io.in2.poke(in2.U)
                    dut.io.out.expect((in1 + in2).U)
                }
            }
    }
    true
}

assert(testSum)

## Problem 5 - Seq addition
> Use Scala `zip` and `map` to add the contents of two `Seq`s (element by element).

In [ ]:
def addSeqs(a: Seq[Int], b: Seq[Int]): Seq[Int] = {
    // YOUR CODE HERE
    ???
}

In [ ]:
val a = Seq.tabulate(8)(_.toInt)
val b = Seq.tabulate(8)(_.toInt)
assert(addSeqs(a, b) == Seq(0, 2, 4, 6, 8, 10, 12, 14))


## Problem 6  - foreach with Chisel
> The `VecRotate` module below shifts its input `Vec` by a constant `offset` (wraps around). Complete it by using `foreach`. You may find the Scala methods `drop` and `take` helpful.

In [ ]:
class VecRotate(numElems: Int, width: Int, offset: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(numElems, UInt(width.W)))
        val out = Output(Vec(numElems, UInt(width.W)))
    })
    val rotated = io.in.drop(offset) ++ io.in.take(offset)
    // YOUR CODE HERE
    ???
}

In [ ]:
def testVecRotate(numElems: Int, width: Int): Boolean = {
    for (offset <- 0 until numElems) {
        val input = 0 until numElems
        val expected = input.drop(offset) ++ input.take(offset)
        test(new VecRotate(numElems, width, offset)) { dut =>
            (0 until numElems).foreach{ i => dut.io.in(i).poke(input(i).U) }
            (0 until numElems).foreach{ i => dut.io.out(i).expect(expected(i).U) }
        }
    }
    true
}

assert(testVecRotate(4,8))

## Problem 7  - zipWithIndex
> First, use `foldLeft` to implement the `exp` function (computes exponent). Then use `zipWithIndex`, `map`, `exp`, and `reduce`/`foldLeft` to concisely evaluate a polynomial. The index in the sequence is the degree in the polynomial (e.g. _coefs(i) * x^i_)

In [ ]:
def exp(base: Int, deg: Int): Int = {
    // YOUR CODE HERE
    ???
}

def polyEval(coefs: Seq[Int], x: Int): Int = {
    // YOUR CODE HERE
    ???
}

In [ ]:
assert (exp(5, 0) == 1)
assert (exp(2, 5) == 32)
assert (exp(4, 3) == 64)
// 0*x^0 + 1*x^1 + 2*x^2
assert(polyEval(Seq(0, 1, 2), 5) == 55)
assert(polyEval(Seq(0, 1, 2), 0) == 0)


## Problem 8 - map on matrix
> Given a `n` x `n` matrix (`Seq[Seq[Int]]`), use `map` and `zipWithIndex` to add `x` to the diagonal (other cells unchanged). For this function `incDiag`, the matrix is in row-major order. For example if `x=4`: 
``` 
    List(1, 1, 1, 1, 1) -> List(5, 1, 1, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 5, 1, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 5, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 1, 5, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 1, 1, 5)
```

In [ ]:
// YOUR CODE HERE
???

In [ ]:
val in = List(
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1)
)

val out = List(
  List(5, 1, 1, 1, 1),
  List(1, 5, 1, 1, 1),
  List(1, 1, 5, 1, 1),
  List(1, 1, 1, 5, 1),
  List(1, 1, 1, 1, 5)
)
assert(incDiag(in, 4) == out)



## Problem 9 - flatMap and reduce with Chisel
> Let's put together what we've covered to make a Chisel module. Complete the `MatrixSearch` module below that looks for the input `searchFor` by comparing all of the elements of the input matrix `mat` (2D `Vec`). If (and only if) `searchFor` matches any of the elements in `mat`, the output `found` should be _true_. Your solution should use `flatMap`, `reduce`, and possibly `map`.

In [ ]:
class MatrixSearch(numRows: Int, numCols: Int, width: Int) extends Module {
    require(numRows > 1)
    require(numCols > 1)
    val io = IO(new Bundle {
        val mat = Input(Vec(numRows, Vec(numCols, UInt(width.W))))
        val searchFor = Input(UInt(width.W))
        val found = Output(Bool())
    })
    // YOUR CODE HERE
    ???
}

In [ ]:
def testMatrixSearch(numRows: Int, numCols: Int, width: Int): Boolean = {
    require(log2Ceil(numRows) < width)
    test(new MatrixSearch(numRows, numCols, width)) { dut =>
        (0 until numRows) foreach {
             r => (0 until numCols) foreach { 
                c => dut.io.mat(r)(c).poke(r.U)
            }
        }
        for (r <- 0 until numRows) {
            dut.io.searchFor.poke(r.U)
            dut.io.found.expect(true.B)
        }
        dut.io.searchFor.poke(numRows.U)
        dut.io.found.expect(false.B)
    }
    true
}

assert(testMatrixSearch(2,2,8))
